in retrospect this obviously won't work

In [58]:
import pandas as pd
import pickle
df_19 = pd.read_csv("../data/cv-corpus-19.0-delta-2024-09-13/en/validated.tsv", delimiter="\t")
df_21 = pd.read_csv("../data/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", delimiter="\t")
df_17 = pd.read_csv('../data/cv-corpus-17.0-delta-2024-03-15/en/validated.tsv', delimiter="\t")

embeddings_19 = {}
with open('../data/cv-corpus-19.0-delta-2024-09-13/commonvoice_embeddings.pkl', 'rb') as file:
    embeddings_19 = pickle.load(file)
embeddings_21 = {}
with open('../data/cv-corpus-21.0-delta-2025-03-14/commonvoice_embeddings.pkl', 'rb') as file:
    embeddings_21 = pickle.load(file)
embeddings_17 = {}
with open('../data/cv-corpus-17.0-delta-2024-03-15/commonvoice_embeddings.pkl', 'rb') as file:
    embeddings_17 = pickle.load(file)

In [59]:
y_train= []
X_train= []
for idx, gender in enumerate(df_21.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_train.append(gender)
        X_train.append(embeddings_21[df_21.path.tolist()[idx].split(".")[0]])
for idx, gender in enumerate(df_17.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_train.append(gender)
        X_train.append(embeddings_17[df_17.path.tolist()[idx].split(".")[0]])
# y_test= []
# X_test= []
for idx, gender in enumerate(df_19.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_train.append(gender)
        X_train.append(embeddings_19[df_19.path.tolist()[idx].split(".")[0]])

In [60]:
from concept_erasure import LeaceEraser
import torch
eraser = LeaceEraser.fit(torch.tensor(X_train), torch.tensor([0  if y == "female_feminine" else 1 for y in y_train]))


In [67]:
from transformers import AutoProcessor, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer

model_checkpoint = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_checkpoint)
processor = AutoProcessor.from_pretrained(model_checkpoint)
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_checkpoint)

lm_head = model.lm_head
vocab = processor.tokenizer.vocab.keys()

embeddings_19 = {}
with open('../data/cv_19.pkl', 'rb') as file:
    embeddings_19 = pickle.load(file)
# embeddings_21 = {}
# with open('../data/cv_21.pkl', 'rb') as file:
#     embeddings_21 = pickle.load(file)
embeddings_17 = {}
with open('../data/cv_17.pkl', 'rb') as file:
    embeddings_17 = pickle.load(file)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
import torch.nn as nn
import torch
from evaluate import load
import re
from tqdm import tqdm 
import numpy as np
import torch.optim as optim
corpora = [17, 19]
dfs = [df_17, df_19]
embedding_dicts = [embeddings_17, embeddings_19]
# Create a cross-entropy loss function
criterion = nn.CrossEntropyLoss()
# Choose an optimizer (e.g., SGD)
optimizer = optim.Adam(lm_head.parameters(), lr=0.01)
for df_idx, df in enumerate(dfs):
    paths = df.path.to_list()
    genders = df.gender.to_list()
    sentences = df.sentence.to_list()
    curr_dict = embedding_dicts[df_idx]
    for idx, path in tqdm(enumerate(paths)):
        if genders[idx] in ["male_masculine", "female_feminine"]:
            embedding = curr_dict[path.split(".")[0]]
            debiased_embedding = [eraser(torch.tensor(x)) for x in embedding[0]]
            predictions = lm_head.forward(torch.stack(debiased_embedding))
            predicted_ids = torch.argmax(predictions, dim=-1)
            transcription = processor.batch_decode(predicted_ids)
            prediction = transcription[0].lower()
            prediction = tokenizer(prediction).input_ids
            truth = tokenizer(sentences[idx]).input_ids
            # Calculate loss
            loss = criterion(torch.tensor(prediction), torch.tensor(truth))

            # Backward pass
            optimizer.zero_grad() # Zero the gradients from the previous iteration
            loss.backward()
            # Update the weights and biases
            optimizer.step()

0it [00:00, ?it/s]/var/folders/_l/m75yx49j3q98z5jct8l93xd40000gp/T/ipykernel_52634/2027004869.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  debiased_embedding = [eraser(torch.tensor(x)) for x in embedding[0]]
4it [00:00, 13.02it/s]


RuntimeError: "log_softmax_lastdim_kernel_impl" not implemented for 'Long'

In [ ]:
import torch
from evaluate import load
import re
from tqdm import tqdm 
import numpy as np
# corpora = [17, 19, 21]
# dfs = [df_17, df_19, df_21]
# embedding_dicts = [embeddings_17, embeddings_19, embeddings_21]
corpora = [21]
dfs = [df_21]
embedding_dicts = [embeddings_21]
rows = []
for df_idx, df in enumerate(dfs):
    paths = df.path.to_list()
    genders = df.gender.to_list()
    sentences = df.sentence.to_list()
    curr_dict = embedding_dicts[df_idx]
    for idx, path in tqdm(enumerate(paths)):
        if genders[idx] in ["male_masculine", "female_feminine"]:
            embedding = curr_dict[path.split(".")[0]]
            debiased_embedding = [eraser(torch.tensor(x)) for x in embedding[0]]
            predictions = lm_head.forward(torch.stack(debiased_embedding))
            predicted_ids = torch.argmax(predictions, dim=-1)
            transcription = processor.batch_decode(predicted_ids)
            truth = sentences[idx]
            #clean
            truth = re.sub(r'[^\w\s]', '', truth)
            truth = truth.lower()
            wer = load("wer")
            cer = load("cer")
            prediction = transcription[0].lower()
            wer_score = wer.compute(predictions=[prediction], references=[truth])
            cer_score = cer.compute(predictions=[prediction], references=[truth])
            rows.append([path, genders[idx], prediction, truth, wer_score, cer_score, corpora[df_idx]])
df = pd.DataFrame(rows, columns=["path", "gender", "prediction", "truth", "wer", "cer", "cv_corpus"])
df.to_csv("predictions_debiased.csv", index=False)

0it [00:00, ?it/s]/var/folders/_l/m75yx49j3q98z5jct8l93xd40000gp/T/ipykernel_52634/595765433.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  debiased_embedding = [eraser(torch.tensor(x)) for x in embedding[0]]
249it [04:31,  1.09s/it]


In [ ]:
df_female = df[df["gender"] == "female_feminine"]
df_male = df[df["gender"] == "male_masculine"]
df_male.wer.describe(), df_male.cer.describe()


(count    42.0
 mean      1.0
 std       0.0
 min       1.0
 25%       1.0
 50%       1.0
 75%       1.0
 max       1.0
 Name: wer, dtype: float64,
 count    42.000000
 mean      0.959243
 std       0.021865
 min       0.850000
 25%       0.957294
 50%       0.964901
 75%       0.970149
 max       0.978723
 Name: cer, dtype: float64)

In [ ]:
df_female.wer.describe(), df_female.cer.describe()

(count    143.0
 mean       1.0
 std        0.0
 min        1.0
 25%        1.0
 50%        1.0
 75%        1.0
 max        1.0
 Name: wer, dtype: float64,
 count    143.000000
 mean       0.961026
 std        0.015491
 min        0.904762
 25%        0.953125
 50%        0.963636
 75%        0.971831
 max        1.000000
 Name: cer, dtype: float64)